### Config

In [50]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_PROJECT'] = "BuffetBot"
os.environ['GOOGLE_API_KEY'] = os.getenv('GOOGLE_API_KEY')
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

### Knowledge Base (Don't touch)

In [56]:
import os
import numpy as np
from langchain.embeddings import OllamaEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


pdf_files = ["charlie_munger.pdf", "intelligent_investor.pdf", "buffet_essays.pdf"]

documents = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    documents.extend(loader.load())


In [55]:
len(documents)

1259

In [58]:
from uuid import uuid4



index = faiss.IndexFlatL2(len(OpenAIEmbeddings().embed_query("hello world")))

vector_store = FAISS.load_local(
    "knowledge_base", embeddings, allow_dangerous_deserialization=True
)


uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, id=uuids)


['0101e266-97db-4898-a1d9-ee38642d9f39',
 '35b9fe0f-6ae3-49a8-b483-e80ee515c13d',
 '1bd3e775-67c8-40e1-8219-da9265f99922',
 'b8ebe93a-7fb5-4e17-ae23-845a1a7d17e6',
 '0bb08e1b-6c6c-4601-8506-d567d98b5650',
 'b9db24dd-d1d3-4c1d-a4f1-f8e92956b324',
 'b4953f49-10eb-447f-86cf-254efe40623f',
 '05ad67a0-32d4-41cf-a6b4-a2ddf6df4f90',
 '33a7612a-234c-49a5-8c5d-29bba0650455',
 'a2681dd4-f2ce-4828-bf1f-40bad6769ba5',
 'e7811bd4-4346-44e9-961b-7def777f0f9b',
 'eeba63b9-62b8-48cb-9493-7f2b3c3f5595',
 'f473a51d-0475-4f94-ab56-2ef49125ceb1',
 'a2e27502-edbb-4ce5-aab4-a4fa9e24a083',
 '98e05966-fd74-4e0a-bbd4-345307352dfd',
 '610d12b1-5f2a-4e5c-b573-896cf7f61e9d',
 '773d93b5-798a-4e4a-9e82-d6fa48157a5b',
 '7f8b0baa-11e1-42f7-a319-819ee2e1b1e3',
 'ea494795-1db1-4a0b-880e-7f8669b1eead',
 'b6ca6548-d06d-486e-8bae-73a1d0d788b2',
 '8f39e4d6-3078-4643-bc8e-9722a135c1ab',
 'a522d3e8-017f-467d-a0d8-63f6fca30e91',
 '967ac64f-6a02-496b-811e-6a41d7cb858d',
 '60f2d12c-136e-4c45-a70c-5991253fb785',
 '156209f4-22b5-

In [59]:
vector_store.save_local("knowledge_base")

### Playground

In [ ]:
%pip install -r requirements.txt

In [ ]:
import os
import numpy as np
from langchain.embeddings import OllamaEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
os.getenviron['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY') ### PLACE YOUR OPENAI API KEY HERE

In [63]:
embeddings = OpenAIEmbeddings()

vector_store = FAISS.load_local(
    "knowledge_base", embeddings, allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={"k": 5})


### Naive RAG

In [69]:

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
naive_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#### RAG Fusion

In [73]:
from langchain.prompts import ChatPromptTemplate

#------------------------------
# RAG-Fusion Question Generating Prompt
template = """You are a financial assistant that takes questions for Warren Buffet and generates multiple queries, so that Warren understands them properly \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
#------------------------------

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [74]:

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [78]:
generate_queries.invoke("how to invest like you")

['1. What investment strategies does Warren Buffet recommend for long-term success?',
 '2. How does Warren Buffet evaluate potential investment opportunities?',
 '3. What are the key principles that Warren Buffet follows when making investment decisions?',
 "4. How can individual investors apply Warren Buffet's value investing approach to their own portfolios?"]

In [80]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60, n=5):

    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results[:5]

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

5

In [81]:
from langchain_core.runnables import RunnablePassthrough

#------------------------------
# Prompt for Generating Answer from RAG-Fusion. Keep {context} and {question} to not break it.
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
#------------------------------
prompt = ChatPromptTemplate.from_template(template)

rag_fusion_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": RunnablePassthrough()} 
    | prompt
    | llm
    | StrOutputParser()
)

#### HyDE

In [82]:
from langchain.prompts import ChatPromptTemplate

#------------------------------
# Prompt for generating HyDE paragraph
template = """You are Warren Buffet. Answer this question with a passage using your principles: {question}
Passage:"""
#------------------------------


prompt_hyde = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

generate_docs_for_retrieval = (
    prompt_hyde | ChatOpenAI(temperature=0) | StrOutputParser() 
)

# Run
question = "how to invest like you?"
generate_docs_for_retrieval.invoke({"question":question})

"Investing like Warren Buffet requires a disciplined approach and a long-term perspective. One of my key principles is to invest in companies with strong competitive advantages and a proven track record of success. I believe in focusing on the fundamentals of a business, rather than trying to time the market or chase short-term gains.\n\nAnother important aspect of investing like me is to do thorough research and analysis before making any investment decisions. I always look for companies with a solid balance sheet, consistent earnings growth, and a management team that is aligned with shareholders' interests.\n\nI also believe in the power of patience and staying true to your investment thesis, even when the market is volatile or uncertain. By taking a long-term view and staying focused on the fundamentals, I have been able to achieve consistent returns over the years.\n\nIn summary, to invest like Warren Buffet, focus on companies with strong competitive advantages, do your research,

In [91]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retireved_docs = retrieval_chain.invoke({"question":question})
retireved_docs

#------------------------------
# Prompt for Generating Answer from HyDE. Keep {context} and {question} to not break it.
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
#------------------------------

prompt = ChatPromptTemplate.from_template(template)

hyde_rag_chain = (
    {"context": retrieval_chain, 
     "question": RunnablePassthrough()} 
    |prompt
    | llm
    | StrOutputParser()
)

### Testing

In [84]:
question = "how do I invest like you?"

In [86]:
naive_rag_chain.invoke(question)

'To invest like an intelligent investor, focus on controlling what you can, such as minimizing costs, managing risk, and maintaining realistic expectations. Adopt a long-term strategy by consistently investing in a total stock-market index fund and avoid reacting to market fluctuations. Lastly, prioritize self-discipline and continuous learning to improve your investment decisions over time.'

In [87]:
rag_fusion_chain.invoke(question)

'To invest like Warren Buffett, consider the following key principles based on his investment philosophy:\n\n1. **Understand the Business**: Focus on companies that you can easily understand. Buffett emphasizes investing in businesses with strong economic characteristics and a clear business model.\n\n2. **Evaluate Management**: Assess the quality of the management team. Look for leaders who have a track record of making wise decisions, allocating capital effectively, and prioritizing shareholder interests over personal gain.\n\n3. **Look for "Franchise" Companies**: Invest in companies with strong consumer brands, robust financial health, and a competitive edge in their market. These are often companies that can sustain growth over the long term.\n\n4. **Margin of Safety**: Always seek a margin of safety in your investments. This means buying stocks at a price that is significantly below their intrinsic value to minimize risk.\n\n5. **Long-Term Perspective**: Adopt a long-term investm

In [92]:
hyde_rag_chain.invoke({"question":question})

'To invest like the intelligent investors discussed in the provided context, consider the following principles:\n\n1. **Control the Controllable**: Focus on aspects of investing that you can manage, such as:\n   - **Brokerage Costs**: Trade rarely and choose low-cost options.\n   - **Ownership Costs**: Avoid mutual funds with high annual expenses.\n   - **Expectations**: Set realistic return expectations based on historical data rather than fantasies.\n   - **Risk Management**: Decide how much of your total assets to invest in the stock market, diversify your investments, and regularly rebalance your portfolio.\n   - **Tax Efficiency**: Hold investments for at least one year, and ideally five years, to minimize capital gains taxes.\n   - **Behavior**: Maintain discipline and avoid emotional reactions to market fluctuations.\n\n2. **Long-Term Perspective**: Adopt a long-term investment horizon (25-30 years). Consider investing in a total stock-market index fund and contribute regularly,

### SEC API

In [4]:
from edgar_functions import *
ticker = "WSM"

accn = get_filtered_filings(
    ticker, just_accession_numbers=True, headers=headers
)

accn_num = accn.iloc[0].replace("-", "")
label_dict = get_label_dictionary(ticker, headers)
statement = process_one_statement(ticker, accn_num, "income_statement")
rename_statement(statement, label_dict)

,2024-01-28,2023-01-29,2022-01-30
us-gaap_IncomeStatementAbstract,NaN,NaN,NaN
"Revenue from Contract with Customer, Excluding Assessed Tax","7,750,652","8,674,417","8,245,936"
wsm_CostOfGoodsSoldAndOccupancyExpenses,"4,447,051","4,996,684","4,613,973"
Gross Profit,"3,303,601","3,677,733","3,631,963"
"Selling, General and Administrative Expense","2,059,408","2,179,311","2,178,847"
Operating Income (Loss),"1,244,193","1,498,422","1,453,116"
"Interest Income (Expense), Nonoperating, Net","29,162","2,260","-1,865"
"Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest","1,273,355","1,500,682","1,451,251"
Income Tax Expense (Benefit),"323,593","372,778","324,914"
Net Income (Loss) Attributable to Parent,"949,762","1,127,904","1,126,337"
